In [1]:
!pip install kagglehub

In [5]:
import kagglehub
# Download latest version
path = kagglehub.dataset_download("bhavikjikadara/dog-and-cat-classification-dataset")
print("Path to dataset files:", path)

Path to dataset files: /kaggle/input/dog-and-cat-classification-dataset


In [13]:
#check directory name
files = os.listdir('/kaggle/input/dog-and-cat-classification-dataset')
print(files)
dataset_path = '/kaggle/input/dog-and-cat-classification-dataset/PetImages'
file = os.listdir(dataset_path)
print(file)

['PetImages']
['Dog', 'Cat']


In [20]:
#分割数据
import os
import shutil
import random
import numpy as np

original_dataset_path = '/kaggle/input/dog-and-cat-classification-dataset/PetImages'
train_dataset_path = '/kaggle/working/train_set'  # 新的训练集路径
test_dataset_path = '/kaggle/working/test_set'    # 新的测试集路径

# 测试集比例
test_ratio = 0.3

# 确保目标目录存在
os.makedirs(train_dataset_path, exist_ok=True)
os.makedirs(test_dataset_path, exist_ok=True)

# 处理每个类别
for category in ['Cat', 'Dog']:
    # 创建训练和测试目录下的类别子文件夹
    train_category_path = os.path.join(train_dataset_path, category)
    test_category_path = os.path.join(test_dataset_path, category)
    os.makedirs(train_category_path, exist_ok=True)
    os.makedirs(test_category_path, exist_ok=True)

    # 获取类别文件夹中的所有图片
    category_path = os.path.join(original_dataset_path, category)
    files = [f for f in os.listdir(category_path) if os.path.isfile(os.path.join(category_path, f))]

    # 设置随机种子确保可重复性
    random.seed(42)

    # 随机打乱文件列表
    random.shuffle(files)

    # 计算测试集大小
    test_size = int(len(files) * test_ratio)

    # 分割文件列表为训练集和测试集
    test_files = files[:test_size]
    train_files = files[test_size:]

    # 复制测试文件到测试文件夹
    for file in test_files:
        src = os.path.join(category_path, file)
        dst = os.path.join(test_category_path, file)
        try:
            shutil.copy2(src, dst)
        except (IOError, OSError, shutil.Error) as e:
            print(f"无法复制文件 {src} 到 {dst}: {e}")

    # 复制训练文件到训练文件夹
    for file in train_files:
        src = os.path.join(category_path, file)
        dst = os.path.join(train_category_path, file)
        try:
            shutil.copy2(src, dst)
        except (IOError, OSError, shutil.Error) as e:
            print(f"无法复制文件 {src} 到 {dst}: {e}")

    print(f"{category}: 总共 {len(files)} 文件，分割为 {len(train_files)} 训练文件和 {len(test_files)} 测试文件")

print("\n数据集分割完成!")
print(f"训练集路径: {train_dataset_path}")
print(f"测试集路径: {test_dataset_path}")

Cat: 总共 12499 文件，分割为 8750 训练文件和 3749 测试文件
Dog: 总共 12499 文件，分割为 8750 训练文件和 3749 测试文件

数据集分割完成!
训练集路径: /kaggle/working/train_set
测试集路径: /kaggle/working/test_set


In [21]:
#preprocessing train dataset
import tensorflow as tf
from tensorflow.keras.preprocessing.image import ImageDataGenerator
traindatagen = ImageDataGenerator(rescale = 1./255)
trainset = traindatagen.flow_from_directory(train_dataset_path,target_size = (50,50),batch_size = 32,class_mode = 'binary')

Found 17500 images belonging to 2 classes.


In [22]:
#set cnn model structure
import tensorflow as tf
from tensorflow.keras.models import Sequential
from tensorflow.keras import layers,models
from tensorflow.keras.layers import Conv2D,MaxPooling2D,Flatten,Dense

model = models.Sequential()
model.add(layers.Conv2D(32, (3, 3), activation='relu', input_shape=(50, 50, 3)))
model.add(layers.MaxPooling2D((2, 2)))
model.add(layers.Conv2D(32, (3, 3), activation='relu'))
model.add(layers.MaxPooling2D((2, 2)))
model.add(Flatten())
model.add(Dense(128, activation='relu'))
model.add(Dense(1, activation='sigmoid'))


/usr/local/lib/python3.11/dist-packages/keras/src/layers/convolutional/base_conv.py:107: UserWarning: Do not pass an `input_shape`/`input_dim` argument to a layer. When using Sequential models, prefer using an `Input(shape)` object as the first layer in the model instead.
  super().__init__(activity_regularizer=activity_regularizer, **kwargs)


In [23]:
model.compile(optimizer='adam',loss='binary_crossentropy',metrics=['accuracy'])
model.summary()

Model: "sequential_1"

┏━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━┳━━━━━━━━━━━━━━━━━━━━━━━━┳━━━━━━━━━━━━━━━┓
┃ Layer (type)                    ┃ Output Shape           ┃       Param # ┃
┡━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━╇━━━━━━━━━━━━━━━━━━━━━━━━╇━━━━━━━━━━━━━━━┩
│ conv2d_2 (Conv2D)               │ (None, 48, 48, 32)     │           896 │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ max_pooling2d_2 (MaxPooling2D)  │ (None, 24, 24, 32)     │             0 │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ conv2d_3 (Conv2D)               │ (None, 22, 22, 32)     │         9,248 │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ max_pooling2d_3 (MaxPooling2D)  │ (None, 11, 11, 32)     │             0 │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ flatten_1 (Flatten)             │ (None, 3872)           │             0 │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ dense_2 (Dense)                 │ (None, 128)            │       495,744 │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ dense_3 (Dense)                 │ (None, 1)              │           129 │
└─────────────────────────────────┴────────────────────────┴───────────────┘

 Total params: 506,017 (1.93 MB)

 Trainable params: 506,017 (1.93 MB)

 Non-trainable params: 0 (0.00 B)

In [24]:
#train model and give accurancy
model.fit(trainset,epochs=20)

547/547 ━━━━━━━━━━━━━━━━━━━━ 83s 124ms/step - accuracy: 0.9914 - loss: 0.0313
Epoch 16/20
547/547 ━━━━━━━━━━━━━━━━━━━━ 80s 120ms/step - accuracy: 0.9960 - loss: 0.0184
Epoch 17/20
547/547 ━━━━━━━━━━━━━━━━━━━━ 64s 117ms/step - accuracy: 0.9919 - loss: 0.0238
Epoch 18/20
547/547 ━━━━━━━━━━━━━━━━━━━━ 67s 121ms/step - accuracy: 0.9950 - loss: 0.0184
Epoch 19/20
547/547 ━━━━━━━━━━━━━━━━━━━━ 65s 119ms/step - accuracy: 0.9912 - loss: 0.0268
Epoch 20/20
547/547 ━━━━━━━━━━━━━━━━━━━━ 67s 122ms/step - accuracy: 0.9909 - loss: 0.0275


In [25]:
#check accurancy of train and test dateset
accurancy_train = model.evaluate(trainset)
print("train accurancy:",accurancy_train)


506/547 ━━━━━━━━━━━━━━━━━━━━ 3s 76ms/step - accuracy: 0.9984 - loss: 0.0169

/usr/local/lib/python3.11/dist-packages/PIL/TiffImagePlugin.py:950: UserWarning: Truncated File Read
  warnings.warn(str(msg))


547/547 ━━━━━━━━━━━━━━━━━━━━ 41s 75ms/step - accuracy: 0.9984 - loss: 0.0164
train accurancy: [0.010780188255012035, 0.998457133769989]


In [29]:
#check test dataset accurancy
test_set = traindatagen.flow_from_directory(test_dataset_path,target_size = (50,50),batch_size = 32,class_mode = 'binary')
accurancy_test = model.evaluate(test_set)
print("test accurancy:",accurancy_test)

Found 7498 images belonging to 2 classes.
235/235 ━━━━━━━━━━━━━━━━━━━━ 17s 73ms/step - accuracy: 0.7787 - loss: 1.3479
test accurancy: [1.2832635641098022, 0.7839424014091492]


In [ ]:
#test dataset accurancy not pretty high, need imporve